In [ ]:
Notes from Ece:
Deleted invalid States: XX and YY ?
    

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts

# Import API key
from api_keys import g_key

In [2]:
# Read in the Medicare Part D Opioid Prescrimedicare-part-d-opioid-prescriber-summary-file-2016-1.csvber Summary File 2016
provider_data_row = pd.read_csv('medicare-part-d-opioid-prescriber-summary-file-2016-1.csv', encoding='utf-8-sig')
provider_data_row

,NPI,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Specialty Description,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate,Extended-Release Opioid Claims,Extended-Release Opioid Prescribing Rate
0,1003000126,ENKESHAFI,ARDALAN,21502.0,MD,Internal Medicine,545,23.0,4.22%,NaN,NaN
1,1003000142,KHALIL,RASHID,43623.0,OH,Anesthesiology,1733,1004.0,57.93%,63.0,6.27%
2,1003000167,ESCOBAR,JULIO,89403.0,NV,Dentist,49,11.0,22.45%,0.0,0.00%
3,1003000282,BLAKEMORE,ROSIE,37243.0,TN,Nurse Practitioner,146,NaN,NaN,0.0,NaN
4,1003000407,GIRARDI,DAVID,15825.0,PA,Family Practice,2225,17.0,0.76%,0.0,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
1131545,1992999825,DESCHENES,GEOFFREY,98101.0,WA,Otolaryngology,296,28.0,9.46%,0.0,0.00%
1131546,1992999833,SHAW,L. NOAH,10024.0,NY,Psychoanalyst,64,0.0,0.00%,0.0,NaN
1131547,1992999866,COGAN,JACQUELINE,33162.0,FL,Dentist,40,0.0,0.00%,0.0,NaN
1131548,1992999874,JOFFE,GABRIELLA,23116.0,VA,Internal Medicine,192,16.0,8.33%,NaN,NaN


In [26]:
# Limit the data to the fields needed for the analysis. If any missing, the row will be deleted
provider_data = provider_data_row[['NPI', 'NPPES Provider ZIP Code','NPPES Provider State','Specialty Description',
                               'Total Claim Count','Opioid Claim Count','Opioid Prescribing Rate']]

provider_data = provider_data.rename(columns={"NPPES Provider ZIP Code": "Zip_code","NPPES Provider State": "State", 
                              "Specialty Description": "Speciality"})
len(provider_data)
provider_data

,NPI,Zip_code,State,Speciality,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate
0,1003000126,21502.0,MD,Internal Medicine,545,23.0,4.22%
1,1003000142,43623.0,OH,Anesthesiology,1733,1004.0,57.93%
2,1003000167,89403.0,NV,Dentist,49,11.0,22.45%
3,1003000282,37243.0,TN,Nurse Practitioner,146,NaN,NaN
4,1003000407,15825.0,PA,Family Practice,2225,17.0,0.76%
...,...,...,...,...,...,...,...
1131545,1992999825,98101.0,WA,Otolaryngology,296,28.0,9.46%
1131546,1992999833,10024.0,NY,Psychoanalyst,64,0.0,0.00%
1131547,1992999866,33162.0,FL,Dentist,40,0.0,0.00%
1131548,1992999874,23116.0,VA,Internal Medicine,192,16.0,8.33%


In [27]:
# Drop invalid State abbrevations using a list of state abbreviations (51 in total including DC)
provider_data['State'] = provider_data['State'].str.strip()

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Collect data without a valid state abbreviation for counts (11109 total entries)
not_state = provider_data[~provider_data.State.isin(states)]
print(not_state["State"].value_counts())
print(len(not_state["State"]))

# Drop all rows with state abbreviations not in the list "states" in place
provider_data.drop(provider_data[~provider_data.State.isin(states)].index, inplace=True)

# Display cleaned dataframe (from 1131550 rows to 1120441 rows)
provider_data

PR    10377
VI      193
GU      183
ZZ      167
AE       72
AP       53
MP       28
AA       20
XX       12
AS        4
Name: State, dtype: int64
11109


,NPI,Zip_code,State,Speciality,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate
0,1003000126,21502.0,MD,Internal Medicine,545,23.0,4.22%
1,1003000142,43623.0,OH,Anesthesiology,1733,1004.0,57.93%
2,1003000167,89403.0,NV,Dentist,49,11.0,22.45%
3,1003000282,37243.0,TN,Nurse Practitioner,146,NaN,NaN
4,1003000407,15825.0,PA,Family Practice,2225,17.0,0.76%
...,...,...,...,...,...,...,...
1131545,1992999825,98101.0,WA,Otolaryngology,296,28.0,9.46%
1131546,1992999833,10024.0,NY,Psychoanalyst,64,0.0,0.00%
1131547,1992999866,33162.0,FL,Dentist,40,0.0,0.00%
1131548,1992999874,23116.0,VA,Internal Medicine,192,16.0,8.33%


In [44]:
provider_data = provider_data.dropna()
len(provider_data)

812512

In [29]:
# Need to convert "Opioid Prescribing Rate" to number, removing %
provider_data['Opioid Prescribing Rate'] = provider_data['Opioid Prescribing Rate'].str.extract('(\d+)').astype(int)
provider_data["Opioid Prescribing Rate"]

0           4
1          57
2          22
4           0
6          18
           ..
1131544    55
1131545     9
1131546     0
1131547     0
1131548     8
Name: Opioid Prescribing Rate, Length: 812512, dtype: int32

In [30]:
# National Stats
# Opioid Prescribing Rate
# Specialty - top specialties

national_avg = provider_data['Opioid Prescribing Rate'].mean()
national_avg

10.671983921468236

In [31]:
# State level Stats
# Opioid Prescribing Rate


In [32]:
state_group = provider_data.groupby(['State'])
number_provider = state_group['State'].count()
state_avg = state_group['Opioid Prescribing Rate'].mean()

number_provider
state_avg

State
AA    14.666667
AE    16.260870
AK    12.758339
AL    12.092386
AP    17.645161
        ...    
WI    12.232335
WV     8.984302
WY    11.596799
XX    17.125000
ZZ     8.763158
Name: Opioid Prescribing Rate, Length: 61, dtype: float64

In [38]:
# Specialty - top specialties
# Ece: Can we use this in someway?
state_speciality_group = provider_data.groupby(['State','Speciality'])
speciality_state = state_speciality_group['Speciality'].count()
speciality_state

State  Speciality                                                    
AA     Dentist                                                           2
       Diagnostic Radiology                                              1
       Emergency Medicine                                                1
       Family Practice                                                   1
       General Practice                                                  1
                                                                        ..
ZZ     Psychiatry                                                        3
       Psychiatry & Neurology                                            2
       Pulmonary Disease                                                 2
       Specialist                                                        1
       Student in an Organized Health Care Education/Training Program    7
Name: Speciality, Length: 5376, dtype: int64

In [ ]:
# Heatmap of the average prescription by state


In [ ]:
# Find outlier providers at national level, top 10%? How does the data look?

# Pin them on the map as layer

In [ ]:
### Ece: Zip code level provider average for mapping?
### Does that make sense?

In [ ]:
# Merge with death rate by state, 2016, age 55+ (try 2017)